In [17]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib 
matplotlib.use('Agg')
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [18]:
da = pd.read_csv('NHANES.csv')

In [19]:
da.head()

,SEQN,ALQ101,ALQ110,ALQ130,SMQ020,RIAGENDR,RIDAGEYR,RIDRETH1,DMDCITZN,DMDEDUC2,...,BPXSY2,BPXDI2,BMXWT,BMXHT,BMXBMI,BMXLEG,BMXARML,BMXARMC,BMXWAIST,HIQ210
0,83732,1.0,NaN,1.0,1,1,62,3,1.0,5.0,...,124.0,64.0,94.8,184.5,27.8,43.3,43.6,35.9,101.1,2.0
1,83733,1.0,NaN,6.0,1,1,53,3,2.0,3.0,...,140.0,88.0,90.4,171.4,30.8,38.0,40.0,33.2,107.9,NaN
2,83734,1.0,NaN,NaN,1,1,78,3,1.0,3.0,...,132.0,44.0,83.4,170.1,28.8,35.6,37.0,31.0,116.5,2.0
3,83735,2.0,1.0,1.0,2,2,56,3,1.0,5.0,...,134.0,68.0,109.8,160.9,42.4,38.5,37.7,38.3,110.1,2.0
4,83736,2.0,1.0,1.0,2,2,42,4,1.0,4.0,...,114.0,54.0,55.2,164.9,20.3,37.4,36.0,27.2,80.4,2.0


In [20]:
da["SMQ020x"] = da['SMQ020'].replace({1: "Yes", 2: "No", 7: np.nan, 9: np.nan})
da["SMQ020x"]

0       Yes
1       Yes
2       Yes
3        No
4        No
       ... 
5730    Yes
5731     No
5732    Yes
5733    Yes
5734     No
Name: SMQ020x, Length: 5735, dtype: object

In [32]:
da["RIAGENDRx"] = da.RIAGENDR.replace({1: "Male", 2: "Female"})

In [33]:
dx = da[["SMQ020x", "RIAGENDRx"]].dropna()

In [34]:
dx.isna().sum()

SMQ020x      0
RIAGENDRx    0
dtype: int64

In [35]:
pd.crosstab(dx.SMQ020x, dx.RIAGENDRx)

RIAGENDRx,Female,Male
SMQ020x,,
No,2066,1340
Yes,906,1413


In [36]:
dx["SMQ020x"] = dx["SMQ020x"].replace({"Yes": 1, "No": 0})

In [37]:
dx

,SMQ020x,RIAGENDRx
0,1,Male
1,1,Male
2,1,Male
3,0,Female
4,0,Female
...,...,...
5730,1,Female
5731,0,Male
5732,1,Female
5733,1,Male


In [38]:
pd.crosstab(dx.SMQ020x, dx.RIAGENDRx)

RIAGENDRx,Female,Male
SMQ020x,,
0,2066,1340
1,906,1413


In [40]:
dx.value_counts()

SMQ020x  RIAGENDRx
0        Female       2066
1        Male         1413
0        Male         1340
1        Female        906
dtype: int64

In [41]:
#calculating proportion
dz = dx.groupby("RIAGENDRx").agg({"SMQ020x": [np.mean, np.size]})

In [42]:
dz

SMQ020x      
               mean  size
RIAGENDRx                
Female     0.304845  2972
Male       0.513258  2753

In [43]:
dz.columns = ["Proportion", "Total n"]

In [44]:
dz

,Proportion,Total n
RIAGENDRx,,
Female,0.304845,2972
Male,0.513258,2753


### difference of the population proportion

In [58]:
t = 1.96

In [54]:
p_fem = dz.Proportion[0]
n_fem = dz["Total n"][0]

se_fem = np.sqrt(p_fem*(1-p_fem)/n_fem)

In [55]:
p_male = dz.Proportion[1]
n_male = dz["Total n"][1]

se_male = np.sqrt(p_male*(1-p_male)/n_male)

In [56]:
se_diff = np.sqrt(se_male**2 + se_fem**2)

In [57]:
se_diff

0.012729881381407434

In [61]:
d = p_male - p_fem
lcb = d - t * se_diff
ucb = d + t * se_diff

In [62]:
lcb, ucb

(0.18346247413207697, 0.2333636091471941)

In [65]:
b = da.groupby("RIAGENDRx").agg({"BMXBMI": [np.mean, np.std, np.size]})
b

BMXBMI                
                mean       std  size
RIAGENDRx                           
Female     29.939946  7.753319  2976
Male       28.778072  6.252568  2759

In [67]:
b.columns = ["Mean", "Stdev", "Size"]

In [68]:
b

,Mean,Stdev,Size
RIAGENDRx,,,
Female,29.939946,7.753319,2976
Male,28.778072,6.252568,2759


In [75]:
sem_female = b.Stdev[0] / np.sqrt(b.Size[0])
sem_male = b.Stdev[1] / np.sqrt(b.Size[1])

In [74]:
sem_female, sem_male

(0.14212522940758335, 0.11903715722332033)

In [76]:
sem_diff = np.sqrt(sem_female**2 + sem_male**2)

In [77]:

difference = b.Mean[0] - b.Mean[1]

lcb = difference - t * sem_diff  
ucb = difference + t * sem_diff  

In [78]:
lcb, ucb

(0.798509280230502, 1.5252378004234286)

In [81]:
sm.stats.DescrStatsW(da["BMXBMI"].dropna()).zconfint_mean()

(29.1973673315365, 29.567026875457454)

In [ ]:
sm.stats.DescrStatsW(da["BMXBMI"].dropna()).zconfint_mean()

In [ ]:
sm.stats.proportion_confint()